In [ ]:
import os
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('/home/ajai-krishna/work/Phenocam_d3/ndvi/APU_pos_01_2026_01_21_11_53_36.csv',header=None)
df

In [ ]:
ndvi=df.values
plt.figure(figsize=(10, 10))
img = plt.imshow(
    ndvi,
    cmap="RdYlGn",
    vmin=-1,
    vmax=1
)
plt.colorbar(img, label="NDVI")
plt.title("NDVI Map")
plt.xlabel("X (columns)")
plt.ylabel("Y (rows)")
plt.show()

In [ ]:
# df4

mid_col = len(df.columns)//2
mid_row = len(df)//2

df1 = df.loc[ :mid_row-1, :mid_col-1]
df2 = df.loc[mid_row: , :mid_col-1]
df3 = df.loc[ :mid_row-1, mid_col: ]
df4 = df.loc[mid_row: , mid_col: ]
print(np.mean(df1.values))
print(np.mean(df2.values))
print(np.mean(df3.values))
print(np.nanmean(df4.values))

In [ ]:
def grid_statistics(df, n_rows, n_cols, stat):
    """
    Split dataframe into n_rows × n_cols grid
    and compute statistic for each block.
    
    stat options: "mean", "sum", "median", "std"
    """

    row_splits = np.array_split(df.index, n_rows)
    col_splits = np.array_split(df.columns, n_cols)

    results = {}

    for i, rows in enumerate(row_splits):
        for j, cols in enumerate(col_splits):
            block = df.loc[rows, cols]

            if stat == "mean":
                value = block.stack().mean()
            elif stat == "sum":
                value = block.stack().sum()
            elif stat == "median":
                value = block.stack().median()
            elif stat == "std":
                value = block.stack().std()
            else:
                raise ValueError("Invalid stat type")

            results[f"Grid_{i+1}_{j+1}"] = value

    return results

In [ ]:
df_grid_2 = grid_statistics(df,2,2,"mean")
df_grid_2

In [ ]:
values = df_grid_2.values()
labels = [f"G{i+1}" for i in range(len(values))]

plt.figure()
plt.lineplot(labels, values)

plt.xlabel("Grid Blocks")
plt.ylabel("Mean Value")
plt.title("Mean Comparison of Grid Blocks")

plt.show()